In [ ]:
from huggingface_hub import login

login(token="")

In [ ]:
!pip install smolagents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [ ]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [ ]:
# prompt: open all csv files in Cafe+Rewards+Offers

import pandas as pd
import os

def open_csv_files_in_directory(directory_path):
  """
  Opens all CSV files in a given directory and returns a list of DataFrames.

  Args:
    directory_path: The path to the directory containing the CSV files.

  Returns:
    A list of pandas DataFrames, one for each CSV file found.
  """
  dataframes = []
  for filename in os.listdir(directory_path):
    if filename.endswith(".csv") and filename!="data_dictionary.csv":
      print(filename)
      filepath = os.path.join(directory_path, filename)
      try:
        df = pd.read_csv(filepath)
        dataframes.append(df)
      except Exception as e:
        print(f"Error reading file {filename}: {e}")
  return dataframes

# Replace 'Cafe+Rewards+Offers' with the actual directory path
directory_path = '/content/drive/MyDrive/Cafe+Rewards+Offers'
dataframes = open_csv_files_in_directory(directory_path)

# Now you have a list of DataFrames in the `dataframes` variable
# You can access them using indexing, e.g., `dataframes[0]` for the first DataFrame.


offers.csv
events.csv
customers.csv


In [ ]:
df_name=['offers','events','customers']

for i in range(len(dataframes)):
  dataframes[i].to_sql(df_name[i],engine)

In [ ]:
from smolagents import tool


@tool
def sql_engine(query: str) -> str:
    """
    Allows you to perform SQL queries on the table. Returns a string representation of the result.
    The table is named 'receipts'. Its description is as follows:
        Columns:
        - receipt_id: INTEGER
        - customer_name: VARCHAR(16)
        - price: FLOAT
        - tip: FLOAT

    Args:
        query: The query to perform. This should be correct SQL.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output

In [ ]:
updated_description = """Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:"""

inspector = inspect(engine)
for table in ["offers", "events","customers"]:
    columns_info = [(col["name"], col["type"]) for col in inspector.get_columns(table)]

    table_description = f"Table '{table}':\n"

    table_description += "Columns:\n" + "\n".join(
        [f"  - {name}: {col_type}" for name, col_type in columns_info]
    )
    updated_description += "\n\n" + table_description

print(updated_description)

Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:

Table 'offers':
Columns:
  - index: BIGINT
  - offer_id: TEXT
  - offer_type: TEXT
  - difficulty: BIGINT
  - reward: BIGINT
  - duration: BIGINT
  - channels: TEXT

Table 'events':
Columns:
  - index: BIGINT
  - customer_id: TEXT
  - event: TEXT
  - value: TEXT
  - time: BIGINT

Table 'customers':
Columns:
  - index: BIGINT
  - customer_id: TEXT
  - became_member_on: BIGINT
  - gender: TEXT
  - age: BIGINT
  - income: FLOAT


In [ ]:
sql_engine.description = updated_description

agent = CodeAgent(
    tools=[sql_engine],
    model=HfApiModel("Qwen/Qwen2.5-72B-Instruct"),
)



In [ ]:
agent.run("Find the top 5 offers with the highest rewards.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find the top 5 offers with the highest rewards.                                                                 │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = """                                                                                                      
  SELECT offer_id, offer_type, reward                                                                              
  FROM offers                                                                                                      
  ORDER BY reward DESC                                                                                             
  LIMIT 5                                                                                                          
  """                                                                                                              
  top_rewards = sql_engine(query=query)                                                                            
  print(top_rewards)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

('ae264e3637204a6fb9bb56bc8210ddfd', 'bogo', 10)
('4d5c57ea9a6940dd891ad53e9dbe8da0', 'bogo', 10)
('9b98b8c7a33c4b65b9aebfe6a799e6d9', 'bogo', 5)
('0b1e1539f2cc45b7b9fa7c272da2e1d7', 'discount', 5)
('f19421c1d4aa40978ebb69ca19b0e20d', 'bogo', 5)

Out: None

[Step 1: Duration 2.92 seconds| Input tokens: 2,236 | Output tokens: 75]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = """                                                                                                      
  SELECT offer_id, offer_type, reward                                                                              
  FROM offers                                                                                                      
  ORDER BY reward DESC                                                                                             
  LIMIT 5                                                                                                          
  """                                                                                                              
  top_rewards = sql_engine(query=query)                                                                            
  final_answer(top_rewards)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 
('ae264e3637204a6fb9bb56bc8210ddfd', 'bogo', 10)
('4d5c57ea9a6940dd891ad53e9dbe8da0', 'bogo', 10)
('9b98b8c7a33c4b65b9aebfe6a799e6d9', 'bogo', 5)
('0b1e1539f2cc45b7b9fa7c272da2e1d7', 'discount', 5)
('f19421c1d4aa40978ebb69ca19b0e20d', 'bogo', 5)

[Step 2: Duration 3.35 seconds| Input tokens: 4,839 | Output tokens: 162]

"\n('ae264e3637204a6fb9bb56bc8210ddfd', 'bogo', 10)\n('4d5c57ea9a6940dd891ad53e9dbe8da0', 'bogo', 10)\n('9b98b8c7a33c4b65b9aebfe6a799e6d9', 'bogo', 5)\n('0b1e1539f2cc45b7b9fa7c272da2e1d7', 'discount', 5)\n('f19421c1d4aa40978ebb69ca19b0e20d', 'bogo', 5)"

In [ ]:
agent.run("List all offers and the number of times each offer was received by customers.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ List all offers and the number of times each offer was received by customers.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = """                                                                                                      
  SELECT o.offer_id, COUNT(e.customer_id) AS offer_received_count                                                  
  FROM offers o                                                                                                    
  JOIN events e ON e.offer_id = o.offer_id                                                                         
  WHERE e.event = 'offer received'                                                                                 
  GROUP BY o.offer_id                                                                                              
  """                                                                                                              
  result = sql_engine(query)                                                                                       
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'result = sql_engine(query)' due to: OperationalError: (sqlite3.OperationalError) no 
such column: e.offer_id
[SQL: 
SELECT o.offer_id, COUNT(e.customer_id) AS offer_received_count
FROM offers o
JOIN events e ON e.offer_id = o.offer_id
WHERE e.event = 'offer received'
GROUP BY o.offer_id
\]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

[Step 1: Duration 13.85 seconds| Input tokens: 2,240 | Output tokens: 167]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = """                                                                                                      
  SELECT o.offer_id, COUNT(e.customer_id) AS offer_received_count                                                  
  FROM offers o                                                                                                    
  JOIN events e ON e.value LIKE '%offer_id=' || o.offer_id                                                         
  WHERE e.event = 'offer received'                                                                                 
  GROUP BY o.offer_id;                                                                                             
  """                                                                                                              
  result = sql_engine(query)                                                                                       
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:


Out: None

[Step 2: Duration 6.81 seconds| Input tokens: 4,899 | Output tokens: 338]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━